In [1]:
def loadDataSet():
    return [[1,3,4],[1,4,3],[2,3,5],[1,2,3,5],[2,5]]

In [2]:
# 创建C1集合，集合数据为非重复的所有的商品
def createC1(dataSet):
    C1 =[]
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset,C1))

In [3]:
# 根据记录数据D，计算CK(K为C中每个元素的包含个数)中，满足最小支持度的元素
def scanD(D,Ck,minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):  #判断can(Ck中的元素)是否是tid的子集
                if can not in ssCnt.keys():
                    ssCnt[can]=1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))  
    # print(numItems)
    retList = []
    supportData ={}
    # print(ssCnt.keys())
    for key in ssCnt.keys():
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList,supportData

In [4]:
dataSet = loadDataSet()
print(dataSet)
C1 = createC1(dataSet)
print(C1)
D = list(map(set,dataSet))
print(D)
L1,suppData0 = scanD(D,C1,0.5)
print(L1,suppData0)

[[1, 3, 4], [1, 4, 3], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[{1, 3, 4}, {1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]
[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})] {frozenset({1}): 0.6, frozenset({3}): 0.8, frozenset({4}): 0.4, frozenset({2}): 0.6, frozenset({5}): 0.6}


In [5]:
# 通过Lk，和k，生成C(k+1),例如L1和2，生成C2
def aprioriGen(Lk,k):
    retList =[]
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2] #前k-2个项相同时，将两个集合合并，例如{1，3}，{1，4}，合并成{1，3，4}
            L1.sort();L2.sort()
            if L1==L2:
                retList.append(Lk[i] | Lk[j])
    return retList

print("aprioriGen test")
retList= aprioriGen([frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3}), frozenset({1, 4}), frozenset({3, 4})],3)
print(retList)

aprioriGen test
[frozenset({2, 3, 5}), frozenset({3, 4, 5}), frozenset({1, 3, 4})]


In [6]:
def apriori(dataSet,minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set,dataSet))   
    L1,supportData = scanD(D,C1,minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2])>0):
        Ck = aprioriGen(L[k-2],k)  # 通过Lk，和k，生成Ck,例如L1和2，生成C2，
        Lk,supK = scanD(D,Ck,minSupport) #C2通过scanD生成L2，如此循环
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L,supportData

print("apriori test")
L,suppData = apriori(dataSet,minSupport=0.5)
print(L[0])
print(L[1])
print(L[2])
print(suppData)

apriori test
[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]
[frozenset({2, 5}), frozenset({1, 3})]
[]
{frozenset({1}): 0.6, frozenset({3}): 0.8, frozenset({4}): 0.4, frozenset({2}): 0.6, frozenset({5}): 0.6, frozenset({1, 3}): 0.6, frozenset({2, 5}): 0.6, frozenset({3, 5}): 0.4, frozenset({2, 3}): 0.4, frozenset({1, 5}): 0.2, frozenset({1, 2}): 0.2}


In [13]:
def calcConf(freqSet,H,supportData,br1,minConf=0.7):
    print("====== in calcConf:")
    print("freqSet:",freqSet)
    print("H:",H)
    prunedH =[]
    for conseq in H:
        print("conseq:",conseq)
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            br1.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

In [35]:
'''
def rulesFromConseq(freqSet,H,supportData,br1,minConf=0.7):
    print("====== in rulesFromConseq:")
    print("freqSet:",freqSet)
    m = len(H[0])
    if (len(freqSet)>(m+1)):   # 最初进入到该函数，m=1，len(freqSet)=3,则if为True
        print("H:",H)
        Hmp1 = aprioriGen(H,m+1)   
        #生成二元频繁集合，为什么会跳过一元频繁集合? 例如{a,b,c},直接生成{b,c},计算 {a}->{b,c}的可信度，应该先生成{c}，计算{a，b}->{c}
        Hmp1 = calcConf(freqSet, Hmp1,supportData,br1,minConf)
        print("Hmp1:",Hmp1)
        if (len(Hmp1)>1):   #
            rulesFromConseq(freqSet,Hmp1,supportData,br1,minConf)
'''
def rulesFromConseq(freqSet,H,supportData,br1,minConf=0.7):
    print("====== in rulesFromConseq:")
    print("freqSet:",freqSet)
    m = len(H[0])            
    while(len(freqSet)>m):
        print("H:",H)
        H = calcConf(freqSet, H,supportData,br1,minConf)
        if (len(H)>1):
            H = aprioriGen(H,m+1)
            m += 1
        else:
            break
            

In [33]:
def generateRules(L,supportData,minConf=0.7):
    bigRuleList = []
    for i in range(1,len(L)): # 规则不包括一元素频繁集合
        print("\n第{}级频繁集合:{}:".format(i,L[i]))
        for freqSet in L[i]:
            print("频繁集合:",freqSet)
            H1 = [frozenset([item]) for item in freqSet]
            print("H1:",H1)
            if(i>1):  
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf) #三元素及以上频繁集合，调用rulesFromConseq
            else:
                calcConf(freqSet,H1,supportData,bigRuleList,minConf)   #二元素频繁集合，直接计算calcConf
    return bigRuleList

In [37]:
L,suppData = apriori(dataSet,minSupport=0.4)
print("L:",L)
print("suppData",suppData)
rules = generateRules(L,suppData,minConf=0.4)

L: [[frozenset({5}), frozenset({2}), frozenset({4}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3}), frozenset({1, 4}), frozenset({3, 4})], [frozenset({2, 3, 5}), frozenset({1, 3, 4})], []]
suppData {frozenset({1}): 0.6, frozenset({3}): 0.8, frozenset({4}): 0.4, frozenset({2}): 0.6, frozenset({5}): 0.6, frozenset({3, 4}): 0.4, frozenset({1, 4}): 0.4, frozenset({1, 3}): 0.6, frozenset({2, 5}): 0.6, frozenset({3, 5}): 0.4, frozenset({2, 3}): 0.4, frozenset({1, 5}): 0.2, frozenset({1, 2}): 0.2, frozenset({1, 3, 4}): 0.4, frozenset({2, 3, 5}): 0.4}

第1级频繁集合:[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3}), frozenset({1, 4}), frozenset({3, 4})]:
频繁集合: frozenset({2, 3})
H1: [frozenset({2}), frozenset({3})]
====== in calcConf:
freqSet: frozenset({2, 3})
H: [frozenset({2}), frozenset({3})]
conseq: frozenset({2})
frozenset({3}) --> frozenset({2}) conf: 0.5
conseq: frozenset({3})
frozenset({2}) --> frozenset